[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Neuvork/Engeneering-thesis/blob/master/results.ipynb)

In [ ]:
! git clone https://<username>:<password>@github.com/Neuvork/Engeneeringthesis.git --single-branch --branch conv

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [1]:
#DOPISAC CMA
#ZROBIC REKURENCYJNY ES
import matplotlib.pyplot as plt
import numpy as np
from scipy.linalg import sqrtm
import time
from IPython.display import clear_output
import copy
import cupy as cp
import tensorflow_datasets as tfds
import tensorflow as tf
from Engeneeringthesis.sigmas import Sigmas_Neural_Network
from Engeneeringthesis.NeuralNetwork import Neural_Network
from Engeneeringthesis.Cma_es import CMA_ES
from Engeneeringthesis.Caged_CMA_ES import Caged_CMA_ES
from Engeneeringthesis.Logs import Logs
from tensorflow.keras.models import Model
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import mnist

In [ ]:
!nvidia-smi

In [3]:
mempool = cp.get_default_memory_pool()
pinned_mempool = cp.get_default_pinned_memory_pool()
def cuda_memory_clear():
    mempool.free_all_blocks()
    pinned_mempool.free_all_blocks()          

In [4]:
no_debug = 1
basic_debug_mode = 2
super_debug_mode = 3
only_interesting = 5
DEBUG_MODE = only_interesting

In [5]:
train_ds_mnist = tfds.load("mnist", split = "train", shuffle_files=True, batch_size=-1)
test_ds_mnist = tfds.load("mnist", split = "test", shuffle_files=True, batch_size=-1)

train_ds_mnist = tfds.as_numpy(train_ds_mnist)
test_ds_mnist = tfds.as_numpy(test_ds_mnist)

train_ds_mnist = {"image" : cp.array(train_ds_mnist["image"]/255., dtype=cp.float32), "label" : cp.array(train_ds_mnist["label"]) }
test_ds_mnist = {"image" : cp.array(test_ds_mnist["image"]/255., dtype=cp.float32), "label" : cp.array(test_ds_mnist["label"]) }

In [6]:
train_ds_mnist['image'] = train_ds_mnist['image'].reshape((60000, 1, 28, 28))

In [7]:
def evaluate_population(population, train_ds):
    create_input_time = 0
    preds_time = 0
    points_count_time = 0
    j  = 0
    train_dataset = {'image' : train_ds['image'][:int(0.8*len(train_ds['image']))],
                      'label' : train_ds['label'][:int(0.8*len(train_ds['image']))] }
    validation_dataset = {'image' : train_ds['image'][int(0.8*len(train_ds['image'])):],
                      'label' : train_ds['label'][int(0.8*len(train_ds['image'])):] }
    if DEBUG_MODE % basic_debug_mode == 0:
      print("___EVALUATE_POPULATION_START")
    train_scores = cp.zeros(population.population_size, dtype = cp.uint32)
    for image, label in zip(cp.array(train_dataset['image']), cp.array(train_dataset['label'])):
        start = time.time()
        create_input_time += time.time() - start
        start = time.time()
        preds = population.forward(image)
        preds_time += time.time() - start
        start = time.time()
        train_scores += preds == label
        points_count_time += time.time() - start
        j += 1

    validation_scores = cp.zeros(population.population_size, dtype = cp.uint32)
    for image, label in zip(cp.array(validation_dataset['image']), cp.array(validation_dataset['label'])):
        start = time.time()
        create_input_time += time.time() - start
        start = time.time()
        preds = population.forward(image)
        preds_time += time.time() - start
        start = time.time()
        validation_scores += preds == label
        points_count_time += time.time() - start
        j += 1
      
    file = open("scores.txt", "a")
    file.write("\n train classified: " + str(cp.max(train_scores)) 
                + ", train acc: " + str(cp.max(train_scores/len(train_scores))) 
                + ", len: " + str(len(train_scores))
                + ", shape: " + str(train_scores.shape)
                + "\n" )
    file.close()
    return train_scores/len(train_dataset['image']), validation_scores/len(validation_dataset['image'])

In [ ]:
!nvidia-smi

In [9]:
def custom_plot(ax, data):
  XD = np.array([1,2,3,4,5])
  ax.plot(XD)

In [ ]:
logs = Logs([('matrix','covariance'),('vector','sigma'),
                      ('vector','isotropic'),('vector','anisotropic'),('vector','mean'),
                      ('number','best-train-score'), ('number','best-validation-score')])

In [ ]:
logs = Logs([('matrix','covariance'),('population', 'population'),('number','sigma'),
                      ('vector','isotropic'),('vector','anisotropic'),('vector','mean'),
                      ('number','best-train-score'), ('number','best-validation-score'),
                       ('vector', 'mean_act - mena_prev')])

In [12]:
dimensionalities = []
for i in range(784):
  dimensionalities.append(10)
number_of_cages = 784

In [ ]:
POPULATION_SIZE = 64
#input size do zmiany
population = Neural_Network(POPULATION_SIZE,  (1, 28, 28), 
                            [
                             ['conv', (1, 3, 3), [1.,1.]],
                             ['linear', 10, [1.,1.]]
                             ],
                            cage_dimensionalities=np.array(dimensionalities))

In [ ]:
logs = Logs([('matrix','covariance'),('population', 'population'),('number','sigma'),
                      ('vector','isotropic'),('vector','anisotropic'),('vector','mean'),
                      ('number','best-train-score'), ('number','best-validation-score'),
                       ('vector', 'mean_act - mena_prev')])

In [ ]:
classifier = CMA_ES(population, .01, evaluate_population, logs)

In [ ]:
classifier.fit(train_ds_mnist, POPULATION_SIZE//2, POPULATION_SIZE, 4)

In [ ]:
POPULATION_SIZE = 64
#input size do zmiany
population = Neural_Network(POPULATION_SIZE,  (1, 28, 28), 
                            [
                             ['linear', 10, [1.,1.]]
                             ],
                            cage_dimensionalities=np.array(dimensionalities))

In [ ]:
classifier = CMA_ES(population, .01, evaluate_population, logs)
classifier.fit(train_ds_mnist, POPULATION_SIZE//2, POPULATION_SIZE, 4)

In [ ]:
logs = Logs([('matrix','covariance'),('vector','sigma'),
                      ('vector','isotropic'),('vector','anisotropic'),('vector','mean'),
                      ('number','best-train-score'), ('number','best-validation-score')])

In [ ]:
POPULATION_SIZE = 64
#input size do zmiany
population = Neural_Network(POPULATION_SIZE,  (1, 28, 28), 
                            [
                             ['linear', 10, [1.,1.]]
                             ],
                            cage_dimensionalities=np.array(dimensionalities))

In [ ]:
classifier = Caged_CMA_ES(population, .01, evaluate_population, logs, number_of_cages, dimensionalities)
classifier.fit(train_ds_mnist, POPULATION_SIZE//2, POPULATION_SIZE, 4)